In [4]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

In [5]:
# Function to solve the linear programming problem
def solve_lp(U, V, mu):
    n, m = U.shape  # Dimensions of the problem

    # Create the LP problem
    lp_problem = LpProblem("Linear_Programming_Problem", LpMaximize)

    # Decision variables: phi[i][j] for each pair (i, j)
    phi = [[LpVariable(f"phi_{i}_{j}", lowBound=0) for j in range(m)] for i in range(n)]

    # Objective function: Maximize sum(phi[i][j] * V[i][j])
    lp_problem += lpSum(phi[i][j] * V[i][j] for i in range(n) for j in range(m)), "Objective"

    # Constraint 1: sum(phi[i][j]) = mu[i] for all i
    for i in range(n):
        lp_problem += lpSum(phi[i][j] for j in range(m)) == mu[i], f"mu_constraint_{i}"

    # Constraint 2: sum(phi[i][j] * U[i][j]) >= sum(phi[i][j'] * U[i][j']) for all j, j'
    for j in range(m):
        for j_prime in range(m):
            if j != j_prime:
                lp_problem += (
                    lpSum(phi[i][j] * U[i][j] for i in range(n)) >= lpSum(phi[i][j] * U[i][j_prime] for i in range(n)),
                    f"U_constraint_{j}_{j_prime}"
                )

    # Solve the problem
    lp_problem.solve()

    # Extract the solution
    solution = np.array([[phi[i][j].varValue for j in range(m)] for i in range(n)])
    
    max_value = value(lp_problem.objective)

    return solution, max_value

In [6]:
def max_dot_product_columns(U, mu):
    """
    Renvoie le maximum des produits scalaires entre chaque colonne de U et le vecteur mu,
    ainsi que les indices des colonnes qui réalisent ce maximum.

    Args:
        U (numpy.ndarray): Matrice de dimensions (n, m).
        mu (numpy.ndarray): Vecteur de taille n.

    Returns:
        tuple: (max_value, columns) où max_value est le maximum des produits scalaires,
               et columns est la liste des indices des colonnes qui réalisent ce maximum.
    """
    # Vérifier que les dimensions de U et mu correspondent
    n, m = U.shape
    if len(mu) != n:
        raise ValueError("La longueur du vecteur mu doit correspondre au nombre de lignes de U.")
    
    # Calculer le produit scalaire de mu avec chaque colonne de U
    dot_products = [np.dot(U[:, j], mu) for j in range(m)]
    
    # Trouver le maximum des produits scalaires
    max_value = max(dot_products)
    
    # Trouver les indices des colonnes qui réalisent ce maximum
    max_columns = [j for j, value in enumerate(dot_products) if value == max_value]
    
    return max_value, max_columns

def mean_vector_dot_product(indices, V, mu):
    """
    Calcule le produit scalaire entre le vecteur mu et le vecteur moyen
    des colonnes de V dont les indices sont donnés dans la liste 'indices'.

    Args:
        indices (list): Liste des indices des colonnes à sélectionner dans V.
        V (numpy.ndarray): Matrice de dimensions (n, m).
        mu (numpy.ndarray): Vecteur de taille n.

    Returns:
        float: Le produit scalaire entre mu et le vecteur moyen des colonnes sélectionnées.
    """
    # Vérifier que les dimensions de V et mu correspondent
    n, m = V.shape
    if len(mu) != n:
        raise ValueError("La longueur du vecteur mu doit correspondre au nombre de lignes de V.")
    
    # Sélectionner les colonnes de V spécifiées par les indices
    selected_columns = V[:, indices]
    
    # Calculer le vecteur moyen
    mean_vector = np.mean(selected_columns, axis=1)
    
    # Calculer le produit scalaire du vecteur moyen avec mu
    dot_product = np.dot(mean_vector, mu)
    
    return dot_product

In [14]:
# Example input matrices U, V, and vector mu
U = np.array([[1, 0], [0, 1]])  # Replace with your U matrix
V = np.array([[0, 1], [0, 1]])  # Replace with your V matrix
mu = np.array([0.7,0.3]) # Replace with your mu vector

phi, max_value = solve_lp(U, V, mu)

utilitee_esperee_initiale_receiver, actions_optimales_initiales = max_dot_product_columns(U, mu)

utilitee_esperee_initiale_sender = mean_vector_dot_product(actions_optimales_initiales, V, mu)

utilite_esperee_receiver = np.sum(np.array([phi[i,j]*U[i,j] for i in range(U.shape[0]) for j in range (U.shape[1])]))

# Solve the problem


# Display the solution
print("Phi optimal:")
print(phi)
print(f"Utilité espérée du Sender : {max_value}")
print(f"Utilité espérée initiale du Sender : {utilitee_esperee_initiale_sender}")


print(f"Utilité espérée du Receiver : {utilite_esperee_receiver}")
print(f"Utilité espérée initiale du Receiver : {utilitee_esperee_initiale_receiver}")



Phi optimal:
[[0.4 0.3]
 [0.  0.3]]
Utilité espérée du Sender : 0.6
Utilité espérée initiale du Sender : 0.0
Utilité espérée du Receiver : 0.7
Utilité espérée initiale du Receiver : 0.7


In [39]:
U = np.array([
    [ 1,  0, -1, -2],
    [ 0,  1,  0, -1],
    [ 0,  0,  1,  0],
    [ 0,  0,  1,  1],
    [-1,  0,  0,  1]
])

# Matrice d'utilité du Sender (le fournisseur)
V = np.array([
    [0, 1, 2, 3],
    [0, 1, 2, 3],
    [0, 1, 2, 3],
    [0, 1, 2, 3],
    [0, 1, 2, 3]
])

# Probabilité mu sur les états (distribution des niveaux de demande)
mu = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

phi, max_value = solve_lp(U, V, mu)

utilitee_esperee_initiale_receiver, actions_optimales_initiales = max_dot_product_columns(U, mu)

utilitee_esperee_initiale_sender = mean_vector_dot_product(actions_optimales_initiales, V, mu)

utilite_esperee_receiver = np.sum(np.array([phi[i,j]*U[i,j] for i in range(U.shape[0]) for j in range (U.shape[1])]))

# Solve the problem


# Display the solution
print("Phi optimal:")
print(phi)
print(f"Utilité espérée du Sender : {max_value}")
print(f"Utilité espérée initiale du Sender : {utilitee_esperee_initiale_sender}")


print(f"Utilité espérée du Receiver : {utilite_esperee_receiver}")
print(f"Utilité espérée initiale du Receiver : {utilitee_esperee_initiale_receiver}")


Phi optimal:
[[0.  0.  0.1 0.1]
 [0.  0.  0.1 0.1]
 [0.  0.  0.2 0. ]
 [0.  0.  0.  0.2]
 [0.  0.  0.  0.2]]
Utilité espérée du Sender : 2.6
Utilité espérée initiale du Sender : 1.5000000000000002
Utilité espérée du Receiver : 0.2
Utilité espérée initiale du Receiver : 0.2
